In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re

In [44]:
df=pd.read_csv('data/TenderResultNotice.csv')

In [45]:
df.head()

,clientType,orderType,tenderType,noticeType,noticeNumber,bzpNumber,isTenderAmountBelowEU,publicationDate,orderObject,cpvCode,...,organizationName,organizationCity,organizationProvince,organizationCountry,organizationNationalId,organizationId,tenderId,htmlBody,contractors,objectId
0,1.5,Services,1.1.1,TenderResultNotice,2023/BZP 00000008/01,2023/BZP 00000008,True,2023-01-01T16:42:33.371Z,„Przewóz dzieci przedszkolnych i uczniów szkół...,60100000-9 (Usługi w zakresie transportu drogo...,...,Tarnogórski Ośrodek Sportu i Rekreacji Spółka ...,Tarnowskie Góry,PL24,PL,6450203947,9111,ocds-148610-7e74b48f-5f85-11ed-8832-4e4740e186ac,"<html><head><meta charset=""UTF-8""><style>body ...","[{'contractorName': None, 'contractorCity': No...",08daec17-2e71-d42c-e8e1-8400116e04b1
1,1.5,Services,1.1.1,TenderResultNotice,2023/BZP 00000010/01,2023/BZP 00000010,True,2023-01-01T17:45:19.737Z,Usługi restauracyjne na rzecz uczniów Szkoły P...,55523100-3 (Usługi w zakresie posiłków szkolny...,...,SZKOŁA PODSTAWOWA IM. WANDY CHOTOMSKIEJ W KIEŁ...,Kiełczów,PL02,PL,9111102754,23977,ocds-148610-0db58845-6b11-11ed-abdb-a69c1593877c,"<html><head><meta charset=""UTF-8""><style>body ...","[{'contractorName': 'HORYZONTY SMAKU S.C', 'co...",08daec1f-f35f-2e95-e8e1-8400116e04b2
2,1.1.5,Services,1.4.1.5,TenderResultNotice,2023/BZP 00000022/01,2023/BZP 00000022,True,2023-01-02T06:04:08.875Z,Zimowe utrzymanie obiektów drogowych: chodnikó...,90620000-9 (Usługi odśnieżania),...,Miejski Zarząd Ulic i Mostów w Tychach,Tychy,PL24,PL,6462447307,2801,ocds-148610-e5149d4c-7d37-11ed-b4ea-f64d350121d2,"<html><head><meta charset=""UTF-8""><style>body ...",[{'contractorName': 'BIO-EKOS Sp. z o.o. sp. k...,08daec87-2999-2675-e8e1-8400116e04be
3,1.1.2,Works,1.1.1,TenderResultNotice,2023/BZP 00000031/01,2023/BZP 00000031,True,2023-01-02T06:26:11.022Z,Remont 5 lokali mieszkalnych zasobu Gminy Miej...,"45453000-7 (Roboty remontowe i renowacyjne),45...",...,Zarząd Budynków Komunalnych w Krakowie,Kraków,PL12,PL,6751000311,2769,ocds-148610-406a4c6f-600e-11ed-9171-f6b7c7d59353,"<html><head><meta charset=""UTF-8""><style>body ...","[{'contractorName': 'M-BUD Monika Duda', 'cont...",08daec8a-3da8-b869-100a-0600107fa4a8
4,1.4,Services,1.1.1,TenderResultNotice,2023/BZP 00000037/01,2023/BZP 00000037,True,2023-01-02T06:32:06.056Z,Świadczenie usług utrzymania czystości.,"90919200-4 (Usługi sprzątania biur),90919100-3...",...,Centrum Projektów Europejskich,Warszawa,PL14,PL,7010158887,7272,ocds-148610-1ec2c002-7487-11ed-b4ea-f64d350121d2,"<html><head><meta charset=""UTF-8""><style>body ...","[{'contractorName': None, 'contractorCity': No...",08daec8b-1147-5014-100a-0600107fa4ad


In [55]:
def remove_none(cont):
    cont=eval(cont)
    return len([x for x in cont if not all(v is None for v in x.values())])

In [59]:
df['contractors']=df['contractors'].apply(remove_none)

In [60]:
df.head()

,clientType,orderType,tenderType,noticeType,noticeNumber,bzpNumber,isTenderAmountBelowEU,publicationDate,orderObject,cpvCode,...,organizationName,organizationCity,organizationProvince,organizationCountry,organizationNationalId,organizationId,tenderId,htmlBody,contractors,objectId
0,1.5,Services,1.1.1,TenderResultNotice,2023/BZP 00000008/01,2023/BZP 00000008,True,2023-01-01T16:42:33.371Z,„Przewóz dzieci przedszkolnych i uczniów szkół...,60100000-9 (Usługi w zakresie transportu drogo...,...,Tarnogórski Ośrodek Sportu i Rekreacji Spółka ...,Tarnowskie Góry,PL24,PL,6450203947,9111,ocds-148610-7e74b48f-5f85-11ed-8832-4e4740e186ac,"<html><head><meta charset=""UTF-8""><style>body ...",2,08daec17-2e71-d42c-e8e1-8400116e04b1
1,1.5,Services,1.1.1,TenderResultNotice,2023/BZP 00000010/01,2023/BZP 00000010,True,2023-01-01T17:45:19.737Z,Usługi restauracyjne na rzecz uczniów Szkoły P...,55523100-3 (Usługi w zakresie posiłków szkolny...,...,SZKOŁA PODSTAWOWA IM. WANDY CHOTOMSKIEJ W KIEŁ...,Kiełczów,PL02,PL,9111102754,23977,ocds-148610-0db58845-6b11-11ed-abdb-a69c1593877c,"<html><head><meta charset=""UTF-8""><style>body ...",1,08daec1f-f35f-2e95-e8e1-8400116e04b2
2,1.1.5,Services,1.4.1.5,TenderResultNotice,2023/BZP 00000022/01,2023/BZP 00000022,True,2023-01-02T06:04:08.875Z,Zimowe utrzymanie obiektów drogowych: chodnikó...,90620000-9 (Usługi odśnieżania),...,Miejski Zarząd Ulic i Mostów w Tychach,Tychy,PL24,PL,6462447307,2801,ocds-148610-e5149d4c-7d37-11ed-b4ea-f64d350121d2,"<html><head><meta charset=""UTF-8""><style>body ...",1,08daec87-2999-2675-e8e1-8400116e04be
3,1.1.2,Works,1.1.1,TenderResultNotice,2023/BZP 00000031/01,2023/BZP 00000031,True,2023-01-02T06:26:11.022Z,Remont 5 lokali mieszkalnych zasobu Gminy Miej...,"45453000-7 (Roboty remontowe i renowacyjne),45...",...,Zarząd Budynków Komunalnych w Krakowie,Kraków,PL12,PL,6751000311,2769,ocds-148610-406a4c6f-600e-11ed-9171-f6b7c7d59353,"<html><head><meta charset=""UTF-8""><style>body ...",5,08daec8a-3da8-b869-100a-0600107fa4a8
4,1.4,Services,1.1.1,TenderResultNotice,2023/BZP 00000037/01,2023/BZP 00000037,True,2023-01-02T06:32:06.056Z,Świadczenie usług utrzymania czystości.,"90919200-4 (Usługi sprzątania biur),90919100-3...",...,Centrum Projektów Europejskich,Warszawa,PL14,PL,7010158887,7272,ocds-148610-1ec2c002-7487-11ed-b4ea-f64d350121d2,"<html><head><meta charset=""UTF-8""><style>body ...",1,08daec8b-1147-5014-100a-0600107fa4ad


In [61]:
len(df)

243843

In [64]:
df.drop(np.where(df['contractors']==0)[0], inplace=True)

In [72]:
df.drop_duplicates(inplace=True)

In [73]:
len(df)

201512

In [87]:
df.drop(columns=['clientType', 'tenderType', 'noticeType', 'noticeNumber', 'publicationDate', 'organizationName', 'organizationCity', 'submittingOffersDate', 'procedureResult', 'organizationProvince', 'organizationCountry', 'organizationNationalId', 'organizationId', 'tenderId', 'objectId'], inplace=True)

In [228]:
def remove_tags(html):
    soup=BeautifulSoup(html)
    txt=soup.get_text()
    c={}
    txt=list(txt.split())
    pattern=r"(\d+\.\d+\.?\))"
    num_pattern=r"(\d+\.\d+)"
    for x in range(len(txt)):
    
        if txt[x]=='PLN':
            i=x-1

            p=[]
            while True:
                i-=1
                if re.search(pattern, txt[i]) is not None:
                
                    p.append(txt[i])
                    break
            
            t=' '.join(p)
            cand=txt[x-1]
            if '.' in cand:
                
                cand=cand.replace('.','')
     
            if ',-' in cand:
                cand=cand.replace('-', '0')
      
            if ',' in cand:
                cand=cand.replace(',','.')
     
            cand=re.findall(num_pattern, cand)
            if len(cand)>0:
                cand=float(cand[0])
                c[t]=cand
            
           

    return c

In [225]:
from copy import deepcopy

In [229]:
subset=deepcopy(df[:1000])

In [230]:
subset['htmlBody']=subset['htmlBody'].apply(remove_tags)

In [233]:
subset.to_csv('data/subsetWyniki.csv')

In [214]:
subset.head()

,orderType,bzpNumber,isTenderAmountBelowEU,orderObject,cpvCode,htmlBody,contractors
0,Services,2023/BZP 00000008,True,„Przewóz dzieci przedszkolnych i uczniów szkół...,60100000-9 (Usługi w zakresie transportu drogo...,{},2
1,Services,2023/BZP 00000010,True,Usługi restauracyjne na rzecz uczniów Szkoły P...,55523100-3 (Usługi w zakresie posiłków szkolny...,{},1
2,Services,2023/BZP 00000022,True,Zimowe utrzymanie obiektów drogowych: chodnikó...,90620000-9 (Usługi odśnieżania),{},1
3,Works,2023/BZP 00000031,True,Remont 5 lokali mieszkalnych zasobu Gminy Miej...,"45453000-7 (Roboty remontowe i renowacyjne),45...",{},5
4,Services,2023/BZP 00000037,True,Świadczenie usług utrzymania czystości.,"90919200-4 (Usługi sprzątania biur),90919100-3...",{},1


In [211]:
df['htmlBody']=df['htmlBody'].apply(remove_tags)

KeyboardInterrupt: 